In [1]:
import numpy as np
from scipy.optimize import minimize

def objective_func(x):
    """
    目标函数
    """
    return x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + x[3] ** 2 + x[4] ** 2 + x[5] ** 2 + x[6] ** 2

def penalty_func(x):
    """
    惩罚函数
    """
    g1 = x[0] + x[1] + x[2] - 3
    g2 = x[3] + x[4] + x[5] + x[6] - 4
    return max(0, g1) ** 2 + max(0, g2) ** 2

def aia(objective_func, penalty_func, dim, max_iter, pop_size, clone_size, beta_min, beta_max, gamma):
    """
    AIA算法
    """
    # 种群初始化
    init_pop = np.random.uniform(low=-5, high=5, size=(pop_size, dim)) #变量-5<x<5
    # 免疫克隆、选择、变异
    for i in range(max_iter):
        # 免疫克隆
        clones = np.zeros((pop_size * clone_size, dim))
        for j in range(pop_size):
            beta = beta_min + (beta_max - beta_min) * (i / max_iter) ** gamma
            num_clones = int(clone_size / (1 + np.exp(-beta * objective_func(init_pop[j]))))
            for k in range(num_clones):
                clones[j * clone_size + k] = init_pop[j] + np.random.normal(scale=1, size=dim)
        # 选择
        population = np.vstack((init_pop, clones))
        scores = np.apply_along_axis(lambda x: objective_func(x) + penalty_func(x), 1, population)
        indices = np.argsort(scores)[:pop_size]
        init_pop = population[indices]
        # 变异
        for j in range(pop_size):
            init_pop[j] += np.random.normal(scale=1, size=dim)
            init_pop[j] = np.clip(init_pop[j], -5, 5)
    # 返回最优解
    return minimize(lambda x: objective_func(x) + penalty_func(x), x0=init_pop[0], bounds=[(-5, 5)] * dim).x

x_opt = aia(objective_func, penalty_func, dim=7, max_iter=100, pop_size=50, clone_size=10, beta_min=0.01, beta_max=100, gamma=1)
print('最优解：', x_opt)
print('最优目标函数值：', objective_func(x_opt))


最优解： [ 9.13431508e-09 -1.33447609e-08  4.21397317e-09 -1.78540749e-08
 -1.93311616e-08 -2.81101119e-08  1.19907759e-08]
最优目标函数值： 1.905694818660856e-15
